# Agentes com Humano-no-Loop

Temos um assistente de email que usa um roteador para triagem de emails e então passa o email para o agente gerar uma resposta. Também o avaliamos. Mas confiamos completamente nele para gerenciar nossa caixa de entrada autonomamente? Para uma tarefa tão sensível, humano-no-loop (HITL) é importante! Aqui mostraremos como adicionar um humano-no-loop ao nosso assistente de email para que possamos revisar chamadas de ferramentas específicas.

![overview-img](img/overview_hitl.png)

Vamos mostrar como fazer o grafo *pausar* em pontos específicos e aguardar entrada humana.

![overview-img](img/hitl_schematic.png)

#### Carregar Variáveis de Ambiente

In [ ]:
from dotenv import load_dotenv
load_dotenv("../.env")

## Adicionando HITL ao nosso assistente de email

Vamos adicionar HITL ao nosso assistente de email.

Podemos começar com ferramentas, assim como fizemos antes.

Mas agora, adicionaremos uma nova ferramenta Questão que permite ao assistente fazer uma pergunta ao usuário.

In [ ]:
%load_ext autoreload
%autoreload 2

from typing import Literal
from datetime import datetime
from pydantic import BaseModel

from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

from langgraph.graph import StateGraph, START, END
from langgraph.types import interrupt, Command

from email_assistant.prompts import triage_system_prompt, triage_user_prompt, agent_system_prompt_hitl, default_background, default_triage_instructions, default_response_preferences, default_cal_preferences
from email_assistant.tools.default.prompt_templates import HITL_TOOLS_PROMPT
from email_assistant.schemas import State, RouterSchema, StateInput
from email_assistant.utils import parse_email, format_for_display, format_email_markdown

# Ferramentas do agente
@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Escrever e enviar um email."""
    # Resposta placeholder - em aplicação real enviaria email
    return f"Email enviado para {to} com assunto '{subject}' e conteúdo: {content}"

@tool
def schedule_meeting(
    attendees: list[str], subject: str, duration_minutes: int, preferred_day: datetime, start_time: int
) -> str:
    """Agendar uma reunião no calendário."""
    # Resposta placeholder - em aplicação real verificaria calendário e agendaria
    date_str = preferred_day.strftime("%A, %B %d, %Y")
    return f"Reunião '{subject}' agendada em {date_str} às {start_time} por {duration_minutes} minutos com {len(attendees)} participantes"

@tool
def check_calendar_availability(day: str) -> str:
    """Verificar disponibilidade do calendário para um determinado dia."""
    # Resposta placeholder - em aplicação real verificaria calendário real
    return f"Horários disponíveis em {day}: 9:00, 14:00, 16:00"

@tool
# Isso é novo!
class Question(BaseModel):
      """Pergunta para fazer ao usuário."""
      content: str

@tool
class Done(BaseModel):
      """Email foi enviado."""
      done: bool

# Todas as ferramentas disponíveis para o agente
tools = [
    write_email,
    schedule_meeting,
    check_calendar_availability,
    Question,
    Done,
]

tools_by_name = {tool.name: tool for tool in tools}

# Inicializar o LLM para uso com router / saída estruturada
llm = init_chat_model("gemini-2.5-flash", model_provider="google-genai", temperature=0.0)
llm_router = llm.with_structured_output(RouterSchema)

# Inicializar o LLM, forçando uso de ferramenta (de qualquer ferramenta disponível) para o agente
llm = init_chat_model("gemini-2.5-flash", model_provider="google-genai", temperature=0.0)
llm_with_tools = llm.bind_tools(tools, tool_choice="required")

In [ ]:
from rich.markdown import Markdown
Markdown(HITL_TOOLS_PROMPT)

#### Nó de triagem

Definimos uma função python com nossa lógica de roteamento de triagem, assim como fizemos antes.

Mas, se a classificação for `notify`, queremos interromper o grafo para permitir ao usuário revisar o email!

Então vamos para um novo nó, `triage_interrupt_handler`.

In [ ]:
def triage_router(state: State) -> Command[Literal["triage_interrupt_handler", "response_agent", "__end__"]]:
    """Analyze email content to decide if we should respond, notify, or ignore."""

    # Parse the email input
    author, to, subject, email_thread = parse_email(state["email_input"])
    user_prompt = triage_user_prompt.format(
        author=author, to=to, subject=subject, email_thread=email_thread
    )

    # Create email markdown for Agent Inbox in case of notification
    email_markdown = format_email_markdown(subject, author, to, email_thread)

    # Format system prompt with background and triage instructions
    system_prompt = triage_system_prompt.format(
        background=default_background,
        triage_instructions=default_triage_instructions
    )

    # Run the router LLM
    result = llm_router.invoke(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )

    # Decision
    classification = result.classification

    # Process the classification decision
    if classification == "respond":
        print("📧 Classificação: RESPONDER - Este email requer uma resposta")
        # Next node
        goto = "response_agent"
        # Update the state
        update = {
            "classification_decision": classification,
            "messages": [{"role": "user",
                            "content": f"Respond to the email: {email_markdown}"
                        }],
        }
    elif classification == "ignore":
        print("🚫 Classificação: IGNORAR - Este email pode ser ignorado")
        # Next node
        goto = END
        # Update the state
        update = {
            "classification_decision": classification,
        }

    elif classification == "notify":
        print("🔔 Classificação: NOTIFICAR - Este email contém informações importantes")
        # This is new!
        goto = "triage_interrupt_handler"
        # Update the state
        update = {
            "classification_decision": classification,
        }

    else:
        raise ValueError(f"Invalid classification: {classification}")
    return Command(goto=goto, update=update)

#### Manipulador de Interrupção de Triagem

Se a decisão for `notify` o usuário, interrompemos o grafo!

![overview-img](img/HITL_flow_triage.png)

Para isso, adicionamos um novo nó, `triage_interrupt_handler`, que irá:

1. Mostrar a classificação ao usuário se for `notify`: Passaremos um `dict` para a interrupção que contém nossa classificação.
2. Permitir ao usuário responder à decisão: Projetaremos o código para lidar com o que receberemos de volta do Agent Inbox.

Como você pode ver [aqui](https://github.com/langchain-ai/agent-inbox?tab=readme-ov-file#what-do-the-fields-mean), formatamos nossa interrupção com campos específicos para que possa ser visualizada no Agent Inbox:

* `action_request`: A ação e argumentos para a interrupção com `action` (o nome da ação) e `args` (os argumentos da chamada de ferramenta). Isso é renderizado no Agent Inbox como o cabeçalho principal para o evento de interrupção.
* `config`: Configura quais tipos de interação são permitidos, e elementos de UI específicos para cada um.
* `description`: Deve ser detalhada, e pode ser markdown. Isso será renderizado no Agent Inbox como a descrição

In [ ]:
def triage_interrupt_handler(state: State) -> Command[Literal["response_agent", "__end__"]]:
    """Handles interrupts from the triage step."""

    # Parse the email input
    author, to, subject, email_thread = parse_email(state["email_input"])

    # Create email markdown for Agent Inbox in case of notification
    email_markdown = format_email_markdown(subject, author, to, email_thread)

    # Create messages
    messages = [{"role": "user",
                "content": f"Email to notify user about: {email_markdown}"
                }]

    # Create interrupt that is shown to the user
    request = {
        "action_request": {
            "action": f"Email Assistant: {state['classification_decision']}",
            "args": {}
        },
        "config": {
            "allow_ignore": True,
            "allow_respond": True,
            "allow_edit": False,
            "allow_accept": False,
        },
        # Email to show in Agent Inbox
        "description": email_markdown,
    }

    # Agent Inbox responds with a list of dicts with a single key `type` that can be `accept`, `edit`, `ignore`, or `response`.
    response = interrupt([request])[0]

    # If user provides feedback, go to response agent and use feedback to respond to email
    if response["type"] == "response":
        # Add feedback to messages
        user_input = response["args"]
        # Used by the response agent
        messages.append({"role": "user",
                        "content": f"User wants to reply to the email. Use this feedback to respond: {user_input}"
                        })
        # Go to response agent
        goto = "response_agent"

    # If user ignores email, go to END
    elif response["type"] == "ignore":
        goto = END

    # Catch all other responses
    else:
        raise ValueError(f"Invalid response: {response}")

    # Update the state
    update = {
        "messages": messages,
    }

    return Command(goto=goto, update=update)

#### Chamada LLM

O nó `llm_call` é o mesmo de antes:

In [ ]:
def llm_call(state: State):
    """LLM decides whether to call a tool or not."""

    return {
        "messages": [
            llm_with_tools.invoke(
                [
                    {"role": "system", "content": agent_system_prompt_hitl.format(tools_prompt=HITL_TOOLS_PROMPT,
                                                                                  background=default_background,
                                                                                  response_preferences=default_response_preferences,
                                                                                  cal_preferences=default_cal_preferences)}
                ]
                + state["messages"]
            )
        ]
    }

#### Manipulador de Interrupção

O `interrupt_handler` é o componente HITL central do nosso agente de resposta.

Seu trabalho é examinar as chamadas de ferramenta que o LLM quer fazer e determinar quais precisam de revisão humana antes da execução. Veja como funciona:

1. **Seleção de Ferramenta**: O manipulador mantém uma lista de "ferramentas HITL" que requerem aprovação humana:
   - `write_email`: Já que enviar emails tem impacto externo significativo
   - `schedule_meeting`: Já que agendar reuniões afeta calendários
   - `Question`: Já que fazer perguntas aos usuários requer interação direta

2. **Execução Direta**: Ferramentas não na lista HITL (como `check_calendar_availability`) são executadas imediatamente sem interrupção. Isso permite que operações de baixo risco prossigam automaticamente.

3. **Preparação de Contexto**: Para ferramentas que requerem revisão, o manipulador:
   - Recupera o email original para contexto
   - Formata os detalhes da chamada de ferramenta para exibição clara
   - Configura quais tipos de interação são permitidos para cada tipo de ferramenta

4. **Criação de Interrupção**: O manipulador cria uma solicitação de interrupção estruturada com:
   - O nome da ação e argumentos
   - Configuração para tipos de interação permitidos
   - Uma descrição que inclui tanto o email original quanto a ação proposta

5. **Processamento de Resposta**: Após a interrupção, o manipulador processa a resposta humana:
   - **Accept**: Executa a ferramenta com argumentos originais
   - **Edit**: Atualiza a chamada de ferramenta com argumentos editados e então executa
   - **Ignore**: Cancela a execução da ferramenta
   - **Response**: Registra feedback sem execução

Este manipulador garante que humanos tenham supervisão de todas as ações significativas enquanto permite que operações rotineiras prossigam automaticamente.

A capacidade de editar argumentos de ferramenta (como conteúdo de email ou detalhes de reunião) dá aos usuários controle preciso sobre as ações do assistente.

Podemos visualizar o fluxo geral:

![overview-img](img/HITL_flow.png)

In [ ]:
def interrupt_handler(state: State) -> Command[Literal["llm_call", "__end__"]]:
    """Creates an interrupt for human review of tool calls"""

    # Store messages
    result = []

    # Go to the LLM call node next
    goto = "llm_call"

    # Iterate over the tool calls in the last message
    for tool_call in state["messages"][-1].tool_calls:

        # Allowed tools for HITL
        hitl_tools = ["write_email", "schedule_meeting", "Question"]

        # If tool is not in our HITL list, execute it directly without interruption
        if tool_call["name"] not in hitl_tools:

            # Execute tool without interruption
            tool = tools_by_name[tool_call["name"]]
            observation = tool.invoke(tool_call["args"])
            result.append({"role": "tool", "content": observation, "tool_call_id": tool_call["id"]})
            continue

        # Get original email from email_input in state
        email_input = state["email_input"]
        author, to, subject, email_thread = parse_email(email_input)
        original_email_markdown = format_email_markdown(subject, author, to, email_thread)

        # Format tool call for display and prepend the original email
        tool_display = format_for_display(tool_call)
        description = original_email_markdown + tool_display

        # Configure what actions are allowed in Agent Inbox
        if tool_call["name"] == "write_email":
            config = {
                "allow_ignore": True,
                "allow_respond": True,
                "allow_edit": True,
                "allow_accept": True,
            }
        elif tool_call["name"] == "schedule_meeting":
            config = {
                "allow_ignore": True,
                "allow_respond": True,
                "allow_edit": True,
                "allow_accept": True,
            }
        elif tool_call["name"] == "Question":
            config = {
                "allow_ignore": True,
                "allow_respond": True,
                "allow_edit": False,
                "allow_accept": False,
            }
        else:
            raise ValueError(f"Invalid tool call: {tool_call['name']}")

        # Create the interrupt request
        request = {
            "action_request": {
                "action": tool_call["name"],
                "args": tool_call["args"]
            },
            "config": config,
            "description": description,
        }

        # Send to Agent Inbox and wait for response
        response = interrupt([request])[0]

        # Handle the responses
        if response["type"] == "accept":

            # Execute the tool with original args
            tool = tools_by_name[tool_call["name"]]
            observation = tool.invoke(tool_call["args"])
            result.append({"role": "tool", "content": observation, "tool_call_id": tool_call["id"]})

        elif response["type"] == "edit":

            # Tool selection
            tool = tools_by_name[tool_call["name"]]

            # Get edited args from Agent Inbox
            edited_args = response["args"]["args"]

            # Update the AI message's tool call with edited content (reference to the message in the state)
            ai_message = state["messages"][-1] # Get the most recent message from the state
            current_id = tool_call["id"] # Store the ID of the tool call being edited

            # Create a new list of tool calls by filtering out the one being edited and adding the updated version
            # This avoids modifying the original list directly (immutable approach)
            updated_tool_calls = [tc for tc in ai_message.tool_calls if tc["id"] != current_id] + [
                {"type": "tool_call", "name": tool_call["name"], "args": edited_args, "id": current_id}
            ]

            # Create a new copy of the message with updated tool calls rather than modifying the original
            # This ensures state immutability and prevents side effects in other parts of the code
            # When we update the messages state key ("messages": result), the add_messages reducer will
            # overwrite existing messages by id and we take advantage of this here to update the tool calls.
            result.append(ai_message.model_copy(update={"tool_calls": updated_tool_calls}))

            # Update the write_email tool call with the edited content from Agent Inbox
            if tool_call["name"] == "write_email":

                # Execute the tool with edited args
                observation = tool.invoke(edited_args)

                # Add only the tool response message
                result.append({"role": "tool", "content": observation, "tool_call_id": current_id})

            # Update the schedule_meeting tool call with the edited content from Agent Inbox
            elif tool_call["name"] == "schedule_meeting":


                # Execute the tool with edited args
                observation = tool.invoke(edited_args)

                # Add only the tool response message
                result.append({"role": "tool", "content": observation, "tool_call_id": current_id})

            # Catch all other tool calls
            else:
                raise ValueError(f"Invalid tool call: {tool_call['name']}")

        elif response["type"] == "ignore":
            if tool_call["name"] == "write_email":
                # Don't execute the tool, and tell the agent how to proceed
                result.append({"role": "tool", "content": "User ignored this email draft. Ignore this email and end the workflow.", "tool_call_id": tool_call["id"]})
                # Go to END
                goto = END
            elif tool_call["name"] == "schedule_meeting":
                # Don't execute the tool, and tell the agent how to proceed
                result.append({"role": "tool", "content": "User ignored this calendar meeting draft. Ignore this email and end the workflow.", "tool_call_id": tool_call["id"]})
                # Go to END
                goto = END
            elif tool_call["name"] == "Question":
                # Don't execute the tool, and tell the agent how to proceed
                result.append({"role": "tool", "content": "User ignored this question. Ignore this email and end the workflow.", "tool_call_id": tool_call["id"]})
                # Go to END
                goto = END
            else:
                raise ValueError(f"Invalid tool call: {tool_call['name']}")

        elif response["type"] == "response":
            # User provided feedback
            user_feedback = response["args"]
            if tool_call["name"] == "write_email":
                # Don't execute the tool, and add a message with the user feedback to incorporate into the email
                result.append({"role": "tool", "content": f"User gave feedback, which can we incorporate into the email. Feedback: {user_feedback}", "tool_call_id": tool_call["id"]})
            elif tool_call["name"] == "schedule_meeting":
                # Don't execute the tool, and add a message with the user feedback to incorporate into the email
                result.append({"role": "tool", "content": f"User gave feedback, which can we incorporate into the meeting request. Feedback: {user_feedback}", "tool_call_id": tool_call["id"]})
            elif tool_call["name"] == "Question":
                # Don't execute the tool, and add a message with the user feedback to incorporate into the email
                result.append({"role": "tool", "content": f"User answered the question, which can we can use for any follow up actions. Feedback: {user_feedback}", "tool_call_id": tool_call["id"]})
            else:
                raise ValueError(f"Invalid tool call: {tool_call['name']}")

        # Catch all other responses
        else:
            raise ValueError(f"Invalid response: {response}")

    # Update the state
    update = {
        "messages": result,
    }

    return Command(goto=goto, update=update)

Agora, vamos compilar o grafo.

In [ ]:
from email_assistant.utils import show_graph

# Conditional edge function
def should_continue(state: State) -> Literal["interrupt_handler", "__end__"]:
    """Route to tool handler, or end if Done tool called"""
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        for tool_call in last_message.tool_calls:
            if tool_call["name"] == "Done":
                return END
            else:
                return "interrupt_handler"

# Build workflow
agent_builder = StateGraph(State)

# Add nodes
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("interrupt_handler", interrupt_handler)

# Add edges
agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    {
        "interrupt_handler": "interrupt_handler",
        END: END,
    },
)

# Compile the agent
response_agent = agent_builder.compile()

# Build overall workflow
overall_workflow = (
    StateGraph(State, input=StateInput)
    .add_node(triage_router)
    .add_node(triage_interrupt_handler)
    .add_node("response_agent", response_agent)
    .add_edge(START, "triage_router")

)

email_assistant = overall_workflow.compile()
show_graph(email_assistant, xray=True)

#### Revisão dos Padrões HITL

**Interrupção de Triagem** Quando um email é classificado como "notify", o sistema interrompe para mostrar o email ao usuário humano
- *Decisão do Usuário*: Usuário pode escolher ignorar a notificação ou fornecer feedback para responder ao email
- *Controle de Fluxo*: Se ignorado, o fluxo de trabalho termina; se o usuário fornecer feedback, flui para o Agente de Resposta

**Write Email**: Sistema mostra rascunho de email proposto para revisão humana
- *Decisão do Usuário e Controle de Fluxo*: ignorar (terminar fluxo de trabalho), responder com feedback, aceitar rascunho como está, ou editar rascunho

**Schedule Meeting**: Sistema mostra detalhes de reunião propostos para revisão humana
- *Decisão do Usuário e Controle de Fluxo*: ignorar (terminar fluxo de trabalho), responder com feedback, aceitar detalhes da reunião como estão, ou editar detalhes

**Question**: Sistema faz uma pergunta ao usuário para esclarecer informações
- *Decisão do Usuário e Controle de Fluxo*: ignorar (terminar fluxo de trabalho) ou responder com uma resposta

### Interrupções Nos Permitem Revisar e Aceitar Chamadas de Ferramenta

In [ ]:
import uuid
from langgraph.checkpoint.memory import InMemorySaver

# Email to respond to
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph with checkpointer
checkpointer = InMemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_1 = uuid.uuid4()
thread_config_1 = {"configurable": {"thread_id": thread_id_1}}

# Run the graph until a tool call that we choose to interrupt
print("Executando o grafo até a primeira interrupção...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_1):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

O que aconteceu? Atingimos a [interrupção](https://langchain-ai.github.io/langgraph/concepts/interrupts/), que pausou a execução na chamada de ferramenta. Você pode ver a `action` (nome da chamada de ferramenta) e `args` (argumentos da chamada de ferramenta) que interrompemos exibidos para o usuário.

Agora, como lidamos com a interrupção? É aqui que a interface `Command` entra. [O objeto `Command` tem várias capacidades poderosas](https://langchain-ai.github.io/langgraph/how-tos/command/). Nós o usamos para direcionar o fluxo do grafo em notebooks anteriores:
- `goto`: Especifica qual nó rotear a seguir
- `update`: Modifica o estado antes de continuar a execução

Aqui, vamos usá-lo para retomar o grafo do estado interrompido:
- `resume`: Fornece o valor a retornar da chamada de interrupção

Podemos retornar qualquer valor que nosso grafo está projetado para lidar. No nosso caso, o grafo está projetado para lidar com uma lista de dicts com uma única chave `type` que pode ser `accept`, `edit`, `ignore`, ou `response`. Então, podemos simplesmente passar `{"type": "accept"}` para o argumento `resume` para dizer ao grafo que aceitamos a chamada de ferramenta.

In [ ]:
from langgraph.types import Command

print(f"\nSimulando usuário aceitando a {Interrupt_Object.value[0]['action_request']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_1):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

In [ ]:
print(f"\nSimulando usuário aceitando a {Interrupt_Object.value[0]['action_request']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_1):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

In [ ]:
state = graph.get_state(thread_config_1)
for m in state.values['messages']:
    m.pretty_print()

### Interrupções Nos Permitem Editar Chamadas de Ferramenta

Este teste demonstra como a modificação humana funciona no fluxo HITL:
1. Começamos com o mesmo email de planejamento fiscal de antes
2. O agente propõe uma reunião com os mesmos parâmetros
3. Desta vez, o usuário EDITA a proposta de reunião para mudar:
   - Duração de 45 para 30 minutos
   - Assunto da reunião é tornado mais conciso
4. O agente se adapta a essas mudanças ao rascunhar o email
5. O usuário EDITA ainda mais o email para ser mais curto e menos formal
6. O fluxo de trabalho se completa com ambas as modificações incorporadas

Este cenário mostra um dos aspectos mais poderosos do HITL:

* Usuários podem fazer modificações precisas nas ações do agente antes que sejam executadas, garantindo que o resultado final corresponda às suas preferências sem ter que lidar com todos os detalhes eles mesmos.

In [ ]:
# Same email as before
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph with new thread
checkpointer = InMemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_2 = uuid.uuid4()
thread_config_2 = {"configurable": {"thread_id": thread_id_2}}

# Run the graph until the first interrupt - will be classified as "respond" and the agent will create a write_email tool call
print("Executando o grafo até a primeira interrupção...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_2):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

Edit the `schedule_meeting` tool call

When the agent proposes the initial meeting schedule, we now simulate the user making modifications through the edit functionality. This demonstrates how the `edit` response type works:

1. The user receives the same meeting proposal as in the previous test
2. Instead of accepting, they modify the parameters:
   - Reducing duration from 45 to 30 minutes
   - Keeping the same day and time
3. The `edit` response includes the complete set of modified arguments
4. The interrupt handler replaces the original tool arguments with these edited ones
5. The tool is executed with the user's modifications

This shows how edit capability gives users precise control over agent actions while still letting the agent handle the execution details.

In [ ]:
# Now simulate user editing the schedule_meeting tool call
print("\nSimulando usuário editando a schedule_meeting chamada de ferramenta...")
edited_schedule_args = {
    "attendees": ["pm@client.com", "lance@company.com"],
    "subject": "Tax Planning Discussion",
    "duration_minutes": 30,  # Changed from 45 to 30
    "preferred_day": "2025-05-06",
    "start_time": 14
}

for chunk in graph.stream(Command(resume=[{"type": "edit", "args": {"args": edited_schedule_args}}]), config=thread_config_2):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

Edit the `write_email` tool call

After accepting the modified meeting schedule, the agent drafts an email reflecting the 30-minute duration. Now we demonstrate how editing works with email content:

1. The agent has adapted its email to mention the shorter 30-minute duration
2. We simulate the user wanting an even more significant change to the email:
   - Completely rewriting the content to be shorter and less formal
   - Changing the meeting day mentioned in the email (showing how users can correct agent mistakes)
   - Requesting confirmation rather than stating the meeting as definite
3. The `edit` response contains the complete new email content
4. The tool arguments are updated with this edited content
5. The email is sent with the user's preferred wording

This example shows the power of HITL for complex communication tasks - the agent handles the structure and initial content, while humans can refine tone, style, and substance.

In [ ]:
# Now simulate user editing the write_email tool call
print("\nSimulando usuário editando a write_email chamada de ferramenta...")
edited_email_args = {
    "to": "pm@client.com",
    "subject": "Re: Tax season let's schedule call",
    "content": "Hello Project Manager,\n\nThank you for reaching out about tax planning. I scheduled a 30-minute call next Thursday at 3:00 PM. Would that work for you?\n\nBest regards,\nLance Martin"
}

for chunk in graph.stream(Command(resume=[{"type": "edit", "args": {"args": edited_email_args}}]), config=thread_config_2):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

Look at the full message history, and see trace, to view the edited tool calls:

https://smith.langchain.com/public/21769510-d57a-41e4-b5c7-0ddb23c237d8/r

In [ ]:
state = graph.get_state(thread_config_2)
for m in state.values['messages']:
    m.pretty_print()

### Interrupções Nos Permitem Fornecer Feedback sobre Chamadas de Ferramenta

Este conjunto de testes demonstra a capacidade de "response" - fornecer feedback sem editar ou aceitar:

1. Primeiro, testamos feedback para agendamento de reunião:
   - O usuário fornece preferências específicas (30 minutos em vez de 45, e reuniões à tarde)
   - O agente incorpora este feedback em uma proposta revisada
   - O usuário então aceita o cronograma de reunião revisado

2. Segundo, testamos feedback para rascunho de email:
   - O usuário solicita um email mais curto e menos formal com uma declaração de encerramento específica
   - O agente reescreve completamente o email de acordo com esta orientação
   - O usuário aceita o novo rascunho

3. Por último, testamos feedback para perguntas:
   - Para o convite de brunch, o usuário responde à pergunta com contexto adicional
   - O agente usa essas informações para rascunhar uma resposta de email apropriada
   - O fluxo de trabalho prossegue com a entrada do usuário integrada

A capacidade de "response" preenche a lacuna entre aceitação e edição - usuários podem orientar o agente sem ter que escrever o conteúdo completo eles mesmos. Isso é especialmente poderoso para:
- Ajustar tom e estilo
- Adicionar contexto que o agente perdeu
- Redirecionar a abordagem do agente
- Responder perguntas de uma maneira que molda os próximos passos

In [ ]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph
checkpointer = InMemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_5 = uuid.uuid4()
thread_config_5 = {"configurable": {"thread_id": thread_id_5}}

# Run the graph until the first interrupt
# Email will be classified as "respond"
# Agent will create a schedule_meeting and write_email tool call
print("Executando o grafo até a primeira interrupção...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

Provide feedback for the `schedule_meeting` tool call

Now we explore the feedback capability for meeting scheduling:

1. The agent proposes the standard 45-minute meeting on Tuesday at 2:00 PM
2. Instead of accepting or editing, we provide feedback in natural language
3. Our feedback specifies two preferences:
   - Shorter meeting (30 minutes instead of 45)
   - Preference for afternoon meetings (after 2pm)
4. The agent receives this feedback through the `response` type
5. The interrupt handler adds this feedback as a message to the state
6. The agent processes this feedback and generates a new tool call incorporating these preferences

Unlike direct editing, which requires specifying the entire set of parameters, feedback allows users to express their preferences conversationally. The agent must then interpret this feedback and apply it appropriately to create a revised proposal.

In [ ]:
print(f"\nSimulando usuário fornecendo feedback para a {Interrupt_Object.value[0]['action_request']['action']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "Please schedule this for 30 minutes instead of 45 minutes, and I prefer afternoon meetings after 2pm."}]), config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

Accept the `schedule_meeting` tool call after providing feedback

In [ ]:
print(f"\nSimulando usuário aceitando a {Interrupt_Object.value[0]['action_request']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

Now provide feedback for the `write_email` tool call

After accepting the revised meeting schedule, the agent drafts an email. We now test feedback for email content:

1. The agent's email is relatively formal and detailed
2. We provide stylistic feedback requesting:
   - A shorter, more concise email
   - A less formal tone
   - A specific closing statement about looking forward to the meeting
3. The agent processes this feedback to completely rewrite the email
4. The new draft is much shorter, more casual, and includes the requested closing

This demonstrates the power of natural language feedback for content creation:
- Users don't need to rewrite the entire email themselves
- They can provide high-level guidance on style, tone, and content
- The agent handles the actual writing based on this guidance
- The result better matches user preferences while preserving the essential information

The message history shows both the original and revised emails, clearly showing how the feedback was incorporated.

In [ ]:
print(f"\nSimulando usuário fornecendo feedback para a {Interrupt_Object.value[0]['action_request']['action']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "Shorter and less formal. Include a closing statement about looking forward to the meeting!"}]), config=thread_config_5):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

Accept the `write_email` tool call after providing feedback

In [ ]:
print(f"\nSimulando usuário aceitando a {Interrupt_Object.value[0]['action_request']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

Look at the full message history, and see the trace:

https://smith.langchain.com/public/57006770-6bb3-4e40-b990-143c373ebe60/r

We can see that user feedback in incorporated into the tool calls.  

In [ ]:
state = graph.get_state(thread_config_5)
for m in state.values['messages']:
    m.pretty_print()

### Interrupts Enable New Tools

Now let's try an email that calls the `Question` tool to provide feedback

Finally, we test how feedback works with the `Question` tool:

1. For the brunch invitation email, the agent asks about preferred day and time
2. Instead of ignoring, we provide a substantive response with additional context:
   - Confirming we want to invite the people mentioned
   - Noting we need to check which weekend works best
   - Adding information about needing a reservation
3. The agent uses this information to:
   - Draft a comprehensive email response incorporating all our feedback
   - Notice we didn't provide a specific day/time, so it suggests checking the calendar
   - Include the detail about making a reservation
4. The complete email reflects both the original request and our additional guidance

This demonstrates how question responses can shape the entire workflow:
- Questions let the agent gather missing information
- User responses can include both direct answers and additional context
- The agent integrates all this information into its next actions
- The final outcome reflects the collaborative intelligence of both human and AI

In [ ]:
# Respond
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Partner <partner@home.com>",
    "subject": "Dinner?",
    "email_thread": "Hey, do you want italian or indian tonight?"}

# Compile the graph
checkpointer = InMemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_6 = uuid.uuid4()
thread_config_6 = {"configurable": {"thread_id": thread_id_6}}

# Run the graph until the first interrupt
print("Executando o grafo até a primeira interrupção...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_6):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

Provide feedback for the `Question` tool call

In [ ]:
print(f"\nSimulando usuário fornecendo feedback para a {Interrupt_Object.value[0]['action_request']['action']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "Let's do indian."}]), config=thread_config_6):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

Accept the `write_email` tool call

In [ ]:
print(f"\nSimulando usuário aceitando a {Interrupt_Object.value[0]['action_request']['action']} chamada de ferramenta...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_6):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nOBJETO DE INTERRUPÇÃO:")
        print(f"Solicitação de Ação: {Interrupt_Object.value[0]['action_request']}")

Look at the full message history, and see the trace:

https://smith.langchain.com/public/f4c727c3-b1d9-47a5-b3d0-3451619db8a2/r

We can see that user feedback in incorporated into the email response.

In [ ]:
state = graph.get_state(thread_config_6)
for m in state.values['messages']:
    m.pretty_print()

### Deployment

Let's create a local deployment of our email assistant with HITL from `src/email_assistant/email_assistant_hitl.py`. 
 
As before, run `langgraph dev`, select `email_assistant_hitl` in Studio, and submit the e-mail:

In [ ]:
{
  "author": "Alice Smith <alice.smith@company.com>",
  "to": "John Doe <john.doe@company.com>",
  "subject": "Quick question about API documentation",
  "email_thread": "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
}

Nosso servidor é sem estado. Threads com um deployment local são simplesmente salvas no sistema de arquivos local (`.langgraph_api` na pasta do projeto).

Com um deployment [hospedado](https://langchain-ai.github.io/langgraph/tutorials/deployment/#other-deployment-options), threads são armazenadas em Postgres.

Threads interrompidas são threads com status 'interrupted', e podemos ver a interrupção no Studio:

![studio-img](img/studio-interrupt.png)

Usaremos uma interface customizada para visualizar essas threads interrompidas, [Agent Inbox](https://dev.agentinbox.ai/).

Esta interface é uma boa maneira de editar, aprovar, ignorar, ou fornecer feedback sobre ações específicas tomadas por agentes LangGraph.

Se você for a [dev.agentinbox.ai](https://dev.agentinbox.ai/), pode facilmente se conectar ao grafo:
   * Nome do grafo: o nome do arquivo `langgraph.json` (`email_assistant_hitl`)
   * URL do grafo: `http://127.0.0.1:2024/`

Todas as threads interrompidas executadas serão então visíveis:

![agent-inbox-img](img/agent-inbox.png)

O Agent Inbox simplesmente usa um `Command` com `resume`, como [mostrado com o SDK](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/wait-user-input/#interacting-with-the-agent) acima, para retomar o grafo.